# Workshop: Building an Information Retrieval System for Podcast Episodes

## Objective:
Create an Information Retrieval (IR) system that processes a dataset of podcast transcripts and, given a query, returns the episodes where the host and guest discuss the query topic. Use TF-IDF and BERT for vector space representation and compare the results.

## Instructions:

* Follow the steps outlined above to implement the IR system.
* Run the provided code snippets to understand how each part of the system works.
* Test the system with various queries to observe the results from both TF-IDF and BERT representations.
* Compare and analyze the results. Discuss the pros and cons of each method.
* Document your findings and any improvements you make to the system.

### Step 1: Import Libraries
Import necessary libraries for data handling, text processing, and machine learning.

In [23]:
#Necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


### Step 2: Load the Dataset

Load the dataset of podcast transcripts.

Find the dataset in: https://www.kaggle.com/datasets/rajneesh231/lex-fridman-podcast-transcript

In [24]:
#Load data
podcast_df = pd.read_csv('../WEEK11/archive/podcastdata_dataset.csv')
# Mostrar los nombres de las columnas
print(podcast_df.columns)
# Extract main information in this case is the title and text
titles = podcast_df['title']
text = podcast_df['text']  

# View texxt
print(text.head())

Index(['id', 'guest', 'title', 'text'], dtype='object')
0    As part of MIT course 6S099, Artificial Genera...
1    As part of MIT course 6S099 on artificial gene...
2    You've studied the human mind, cognition, lang...
3    What difference between biological neural netw...
4    The following is a conversation with Vladimir ...
Name: text, dtype: object


In [25]:
# Download necessary resources from NLTK for preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Karen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Karen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Karen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Step 3: Text Preprocessing

You know what to do ;)

In [26]:
# Preprocessing function
def preprocess_text(text):
    # convert lowercase
    text = text.lower()
    # remove special carathers and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens into a single chain
    return ' '.join(tokens)

# Apply preprocessing  to text
podcast_df['processed_text'] = podcast_df['text'].apply(preprocess_text)

# Show some processed texts
print(podcast_df['processed_text'].head())


0    part mit course artificial general intelligenc...
1    part mit course artificial general intelligenc...
2    studied human mind cognition language vision e...
3    difference biological neural network artificia...
4    following conversation vladimir vapnik co inve...
Name: processed_text, dtype: object


###  Step 4: Vector Space Representation - TF-IDF

Create TF-IDF vector representations of the transcripts.

In [27]:

# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustar y transformar las transcripciones procesadas
tfidf_matrix = tfidf_vectorizer.fit_transform(podcast_df['processed_text'])

# Mostrar la forma de la matriz TF-IDF
print("Shape of TF-IDF matrix:", tfidf_matrix.shape)

Shape of TF-IDF matrix: (319, 41079)


In [4]:
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


### Step 5: Vector Space Representation - BERT

Create BERT vector representations of the transcripts using a pre-trained BERT model.

In [28]:

# Cargar el tokenizer y el modelo de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Función para obtener las representaciones BERT
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# Aplicar la función de embeddings BERT a las transcripciones procesadas
podcast_df['bert_embeddings'] = podcast_df['processed_text'].apply(get_bert_embeddings)

# Mostrar la forma de las representaciones BERT
print("Shape of BERT embeddings:", podcast_df['bert_embeddings'].iloc[0].shape)

Shape of BERT embeddings: (768,)


### Step 6: Query Processing

Define a function to process the query and compute similarity scores using both TF-IDF and BERT embeddings.

In [34]:
# Función para procesar la consulta
def process_query(query, podcast_df, tfidf_vectorizer, tfidf_matrix):
    # Preprocesar la consulta
    query_processed = preprocess_text(query)

    # Vectorizar la consulta utilizando TF-IDF
    query_tfidf = tfidf_vectorizer.transform([query_processed])
    
    # Calcular la similitud de coseno para TF-IDF
    tfidf_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    # Vectorizar la consulta utilizando BERT
    query_bert_embedding = get_bert_embeddings(query_processed).reshape(1, -1)
    
    # Calcular la similitud de coseno para BERT
    bert_embeddings = np.vstack(podcast_df['bert_embeddings'].values)
    bert_similarities = cosine_similarity(query_bert_embedding, bert_embeddings).flatten()

    # Crear un DataFrame con los resultados
    results_df = podcast_df.copy()
    results_df['tfidf_similarity'] = tfidf_similarities
    results_df['bert_similarity'] = bert_similarities

    # Ordenar los resultados por la similitud más alta
    results_df = results_df.sort_values(by=['tfidf_similarity', 'bert_similarity'], ascending=False)

    return results_df[['title', 'tfidf_similarity', 'bert_similarity', 'text']]

In [35]:
# Solicitar la entrada del usuario para la consulta
query = input("Ingrese su consulta: ")

# Procesar la consulta del usuario
results = process_query(query, podcast_df, tfidf_vectorizer, tfidf_matrix)

# Mostrar los 5 primeros resultados
print("Query is:",query)
results.head(5)

Query is: following


,title,tfidf_similarity,bert_similarity,text
58,Vsauce,0.019918,0.152706,The following is a conversation with Michael S...
203,String Theory,0.015930,0.137052,The following is a conversation with Kamran Va...
39,iRobot,0.013856,0.165819,The following is a conversation with Colin Ang...
85,Physics of Consciousness and the Infinite Univ...,0.013447,0.148179,The following is a conversation with Roger Pen...
46,"Chess, Deep Blue, AI, and Putin",0.011628,0.147074,The following is a conversation with Gary Kasp...


### Step 7: Retrieve and Compare Results

Define a function to retrieve the top results based on similarity scores for both TF-IDF and BERT representations.

In [37]:
# Función para recuperar y comparar los mejores resultados
def retrieve_and_compare_results(query, podcast_df, tfidf_vectorizer, tfidf_matrix, top_n=5):
    # Procesar la consulta
    results_df = process_query(query, podcast_df, tfidf_vectorizer, tfidf_matrix)

    # Ordenar los resultados por similitud de TF-IDF y recuperar los top N
    top_tfidf_results = results_df.sort_values(by='tfidf_similarity', ascending=False).head(top_n)

    # Ordenar los resultados por similitud de BERT y recuperar los top N
    top_bert_results = results_df.sort_values(by='bert_similarity', ascending=False).head(top_n)

    return top_tfidf_results, top_bert_results

# Recuperar y comparar los resultados principales utilizando la consulta del usuario
top_tfidf_results, top_bert_results = retrieve_and_compare_results(query, podcast_df, tfidf_vectorizer, tfidf_matrix, top_n=5)

print("Query is:",query)
# Mostrar los resultados principales para TF-IDF
print("\nTop TF-IDF Results:")
for idx, row in top_tfidf_results.iterrows():
    print(f"Title: {row['title']}")
    print(f"TF-IDF Similarity: {row['tfidf_similarity']}")
    print(f"Transcript Excerpt: {row['text'][:200]}...")  # Mostrar los primeros 200 caracteres de la transcripción
    print("\n")

# Mostrar los resultados principales para BERT
print("\nTop BERT Results:")
for idx, row in top_bert_results.iterrows():
    print(f"Title: {row['title']}")
    print(f"BERT Similarity: {row['bert_similarity']}")
    print(f"Transcript Excerpt: {row['text'][:200]}...")  # Mostrar los primeros 200 caracteres de la transcripción
    print("\n")

Query is: following

Top TF-IDF Results:
Title: Vsauce
TF-IDF Similarity: 0.019917631536103875
Transcript Excerpt: The following is a conversation with Michael Stevens, the creator of Vsauce, one of the most popular educational YouTube channels in the world with over 15 million subscribers and over 1.7 billion vie...


Title: String Theory
TF-IDF Similarity: 0.01593006918749203
Transcript Excerpt: The following is a conversation with Kamran Valfa, a theoretical physicist at Harvard specializing in string theory. He is the winner of the 2017 Breakthrough Prize in Fundamental Physics, which is th...


Title: iRobot
TF-IDF Similarity: 0.013856277430207535
Transcript Excerpt: The following is a conversation with Colin Angle. He's the CEO and co founder of iRobot, a robotics company that for 29 years has been creating robots that operate successfully in the real world. Not ...


Title: Physics of Consciousness and the Infinite Universe
TF-IDF Similarity: 0.013446839690867485
Transcript Exce

### Step 8: Test the IR System

Test the system with a sample query.

Retrieve and display the top results using both TF-IDF and BERT representations.

In [40]:
# Definir la consulta de prueba
test_query = "child"

# Recuperar y comparar los resultados principales utilizando la consulta de prueba
top_tfidf_results, top_bert_results = retrieve_and_compare_results(test_query, podcast_df, tfidf_vectorizer, tfidf_matrix, top_n=5)

print("Query test is:",test_query)
# Mostrar los resultados principales para TF-IDF
print("\nTop TF-IDF Results:")
for idx, row in top_tfidf_results.iterrows():
    print(f"Title: {row['title']}")
    print(f"TF-IDF Similarity: {row['tfidf_similarity']}")
    print(f"Transcript Excerpt: {row['text'][:200]}...")  # Mostrar los primeros 200 caracteres de la transcripción
    print("\n")

# Mostrar los resultados principales para BERT
print("\nTop BERT Results:")
for idx, row in top_bert_results.iterrows():
    print(f"Title: {row['title']}")
    print(f"BERT Similarity: {row['bert_similarity']}")
    print(f"Transcript Excerpt: {row['text'][:200]}...")  # Mostrar los primeros 200 caracteres de la transcripción
    print("\n")


Query test is: child

Top TF-IDF Results:
Title: Computer Vision
TF-IDF Similarity: 0.12085287681942182
Transcript Excerpt: The following is a conversation with Jitendra Malik, a professor at Berkeley and one of the seminal figures in the field of computer vision, the kind before the deep learning revolution and the kind a...


Title: Yaron Brook and Yoram Hazony
TF-IDF Similarity: 0.06513779225115413
Transcript Excerpt: The following is a conversation with Yoram Brook and Yoram Hazoni. This is Yoram's third time on this podcast and Yoram's first time. Yoram Brook is an Objectivist Philosopher, Chairman of the Ayn Ran...


Title: Hunger, War, and Human Suffering
TF-IDF Similarity: 0.05843336538347112
Transcript Excerpt: we would come up to these rafts and these boats that were in really dire shape and people would be pushed off and people would jump off and people would fall into the water and some of them couldn't s...


Title: Artificial Consciousness and the Nature of Reality
TF-IDF

### Step 9: Compare Results

Analyze and compare the results obtained from TF-IDF and BERT representations.

Discuss the differences, strengths, and weaknesses of each method based on the retrieval results.

TF-IDF 

Ventaja: Los resultados suelen ser relevantes cuando las palabras clave de la consulta aparecen exactamente en el texto.
Desventaja: Puede incluir resultados menos relevantes si las palabras clave son comunes en diferentes contextos.

BERT

Ventaja: Captura mejor el contexto y la semántica, proporcionando resultados más precisos y relevantes.
Desventaja: Los resultados pueden variar y ser menos predecibles, pero suelen tener una comprensión más profunda del tema.


Eficiencia Computacional


TF-IDF: Es más rápido y eficiente computacionalmente. Adecuado para sistemas con recursos limitados.

BERT: Requiere más recursos de procesamiento y memoria, pero ofrece una mayor precisión en la recuperación de información.